In [1]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time

from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC

In [2]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [3]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
#scraped_data_frame = pd.DataFrame() !!!LETS CREATE THE DATA FRAME 
rochelle.head()


,Scientific Name,Taxon,List,GLANSIS Authors,Assessment Documented,Review by:,Shared with USGS,NOAA EndNote Library Status,Assigned to,Action Needed,...,Review Status,Internal NOAA Reviewer(s),USGS Reviewer?,Past External Reviewers,New External Reviewers,Review complete and Cleared to post,Date Profile Last Updated (Posted online),Last Review Date (GLANSIS Records),OIA/RA Posted to Clearinghouse,Twitter
0,Cabomba caroliniana,Plants (S),Established,"Larson, Cao, Berent, Iott",TM161,2021.0,Y,5 year - Received 8/9/2022,EL,None,...,NaN,RS,NaN,Gannon,NaN,NaN,2016-07-25 00:00:00,NaN,NaN,NaN
1,Myriophyllum spicatum,Plants (S),Established,Berent,TM161,2021.0,Y,5 year delivered 3/4/2022,EL,add link to sibiricum x spicatum. Check refer...,...,NaN,NaN,NaN,Dennis Albert,NaN,NaN,2016-03-21 00:00:00,NaN,NaN,NaN
2,Phragmites australis,Plants (E),Established,"Sturtevant, Fusaro, Conard, Iott",TM161b,2021.0,Y,NaN,El,None,...,NaN,NaN,NaN,S. Galatowitsch,NaN,NaN,Complete,NaN,NaN,NaN
3,Nymphoides peltata,Plants (S),Established,Berent,TM161,2021.0,Y,5 year delivered 8/19/2022,EL,Check references are in NAS.,...,NaN,NaN,NaN,Selheimer,NaN,NaN,2016-03-23 00:00:00,NaN,NaN,NaN
4,Potamogeton crispus,Plants (S),Established,"Cao, Berent",TM161,2021.0,Y,5 year delivered 9/13/2022,EL,Check references are in NAS.,...,NaN,NaN,NaN,Selheimer,NaN,NaN,2016-03-18 00:00:00,NaN,NaN,NaN


In [4]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(namelist)

['Cabomba caroliniana', 'Myriophyllum spicatum', 'Phragmites australis', 'Nymphoides peltata', 'Potamogeton crispus', 'Trapa natans', 'Corbicula fluminea', 'Gambusia affinis', 'Pimephales vigilax', 'Crassula helmsii', 'Pistia stratiotes', 'Stratiotes aloides', 'Veronica beccabunga', 'Rhithropanopeus harrisi', 'Thermocyclops crassus', 'Cherax destructor', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lepomis auritus', 'Limnoperna fortunei', 'Alnus glutinosa', 'Apeltes quadracus', 'Arundo donax', 'Branchiura sowerbyi', 'Channa argus', 'Cipangopaludina japonica', 'Cyclops strenuus', 'Didymosphenia geminata', 'Discostella (Cyclotella) pseudostelligera', 'Leuciscus idus', 'Neoergasilus japonicus', 'Procambarus clarkii', 'Psammonobiotus communis', 'Psammonobiotus dziwnowi', 'Psammonobiotus linearis', 'Syngnathus abaster', 'Agrostis gigantea', 'Butomus umbellatus', 'Carassius carassius', 'Carex acutiformis', 'Carex disticha', 'Chenopodium glaucum (Oxybasis glauca)', 'Cirsium p

In [5]:
#rochelle.reset_index(inplace = True,drop = True) 
#name = rochelle.loc[2]["Scientific Name"] #for some reason couldn't access rochelle[0]...
#print(name)


In [6]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [7]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
# url = "https://quod.lib.umich.edu/h/herb00ic/x-1000138/MICH-V-1000138A?lasttype=boolean;lastview=reslist;op2=And;q2=Phragmites+australis+australis;resnum=1;rgn2=ic_all;select2=all;size=50;start=1;subview=detail;view=entry"
#html = urllib.request.urlopen(url, context=ctx).read()
#soup = BeautifulSoup(html, 'html.parser')
#soup

In [8]:
#DEVELOPMENT EXPLORING --- quick learning about using BS with Herbarium
#resp = requests.get(url)
#newsoup = BeautifulSoup(resp.content, 'html.parser')
#iteminfo = soup.find('dl')
#print(iteminfo)

In [9]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [10]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
scraped_dataframe = pd.DataFrame()#(index=[0])
#send driver to the herbarium website and submit a search query and scrape the amount of results
#BE SURE TO SWITCH ALL "FIND ELEMENTS" TO XPATH IF PLAUSIBLE?
for speciesname in namelist:
    driver.get(herbarium_url) # Goes to herbarium
    elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
    elem.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    searchbutton.click()
    #search_results_url = driver.current_url
    resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text
    #next line returns a list of two numbers like ['23','23']
    resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
    resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section[1]/a')
    resultitem.click()

    #individually select each search result and scrape each matching entry

    #driver2 = webdriver.Chrome()
    
    for result in range(int(resultcountnumberlist[0])): #[:-1]: #properly traverses the amount of results on the page
        scrapeddata1 = []
        scrapeddata2 = []
        time.sleep(2) #sleep allows the servers time to communicate
        resp = requests.get(driver.current_url)
        #print(resp.content)
        newsoup = BeautifulSoup(resp.content, 'html.parser')
        collection_category = newsoup.find_all('dt')
        collection_data = newsoup.find_all('dd')
        for tag in collection_category:
            scrapeddata1.append(tag.text)
        for tag in collection_data:
            scrapeddata2.append(tag.text)
        nextbutton = driver.find_element(by=By.XPATH, value='/html/body/main/div[1]/p/a[2]')
        nextbutton.click()
        scraped_data = dict(zip(scrapeddata1, scrapeddata2)) #scraped data includes column names and data
        result_df = pd.DataFrame(data=scraped_data, index=[0])
        #print(result_df)
        if scraped_dataframe.empty:
            scraped_dataframe = result_df
            #print('Empty')
        else:
            scraped_dataframe = scraped_dataframe.append(pd.DataFrame(data = result_df))
            #print('Not Empty Anymore')
        #print(result_df)
    #scraped_dataframe.append(result_df)



ElementClickInterceptedException: Message: Element <a class="[ flex ][ flex-grow-1 ]" href="https://quod.lib.umich.edu/h/herb00ic/x-1102786/MICH-V-1102786?c=fung2ic;c=fuwatic;c=herb00ic;c=herb2ic;c=herb3ic;c=herb4ic;c=herb5ic;c=herb6ic;g=herb-ic;lasttype=boolean;lastview=reslist;resnum=1;size=50;start=1;subview=detail;view=entry;xc=1;rgn1=ic_all;select1=all;q1=Myriophyllum+spicatum"> is not clickable at point (799,612) because another element <nav id="pagination" class="[ pagination__row ][ flex flex-space-between flex-align-center sticky-bottom ]"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:279:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:160:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:119:11
clickElement@c

In [ ]:
scraped_dataframe = scraped_dataframe.reset_index(drop=True)

In [12]:
#scraped_dataframe.columns
print(resultcountnumberlist)

[50, 96]


In [ ]:
#aaa =result_df.columns

In [ ]:
#testdf = pd.DataFrame()
#columns = []
#for c in aaa:

#    if c not in testdf.columns:
#        #print(c)
#        columns.append(c)
#ntestdf = pd.DataFrame(columns=columns)

In [ ]:
#ntestdf.columns

In [ ]:
#ntestdf

In [ ]:
#scrapeddata1